# Exploring Hacker News Posts

In this project, I'll work with a data set of submissions to popular technology site [Hacker News](https://news.ycombinator.com/).

Hacker News is a site started by the startup incubator [Y Combinator](https://www.ycombinator.com/), where user-submitted stories (known as "posts") are voted and commented upon, similar to reddit. Hacker News is extremely popular in technology and startup circles, and posts that make it to the top of Hacker News' listings can get hundreds of thousands of visitors as a result.

I'm specifically interested in posts whose titles begin with either **Ask HN** or **Show HN**. Users submit Ask HN posts to ask the Hacker News community a specific question; Users submit Show HN posts to show the Hacker News community a project, product, or just generally something interesting.

I'll compare these two types of posts to determine the following:

- Do Ask HN or Show HN receive more comments on average?
- Do posts created at a certain time receive more comments on average?

## Opening and Exploring the Data

I'll start by opening and reading the data set so that I can begin managing/manipulating it.

In [1]:
from csv import reader

opened_file = open("../data_sets/hacker_news.csv")
read_file = reader(opened_file)

hn = list(read_file)
hn_header = hn[0]
hn = hn[1:] # Exclude the header row (column names) from the data set

Next, to make the data set easier to explore, I'll create a function named `explore_data()` that I can repeatedly use to print rows of data in a readable way. In addition, when I use this function, I'll have the option to output the number of rows and columns in the data set. After I define this function, I'll print the columns (header row) of the data set to understand what each data point represents. Lastly, I'll use the function to investigate a small selection of the data.

In [2]:
def explore_data(data_set, start, end, rows_and_columns=False):
    data_slice = data_set[start:end]
    for row in data_slice:
        print(row)
        
        # Print an extra line of space after each row for readability
        print()
        
    if rows_and_columns:
        print("Number of rows:", len(data_set))
        print("Number of columns:", len(data_set[0]))
        
print(hn_header)

print()

explore_data(hn, 0, 3, True) # First few rows of the data set

['id', 'title', 'url', 'num_points', 'num_comments', 'author', 'created_at']

['12579008', 'You have two days to comment if you want stem cells to be classified as your own', 'http://www.regulations.gov/document?D=FDA-2015-D-3719-0018', '1', '0', 'altstar', '9/26/2016 3:26']

['12579005', 'SQLAR  the SQLite Archiver', 'https://www.sqlite.org/sqlar/doc/trunk/README.md', '1', '0', 'blacksqr', '9/26/2016 3:24']

['12578997', 'What if we just printed a flatscreen television on the side of our boxes?', 'https://medium.com/vanmoof/our-secrets-out-f21c1f03fdc8#.ietxmez43', '1', '0', 'pavel_lishin', '9/26/2016 3:19']

Number of rows: 293119
Number of columns: 7


You can find the data set [here](https://www.kaggle.com/hacker-news/hacker-news-posts), but note that it has been reduced from almost 300,000 rows to approximately 20,000 rows by removing all submissions that did not receive any comments and then randomly sampling from the remaining submissions. Below are descriptions of the columns:

- `id`: The unique identifier from Hacker News for the post


- `title`: The title of the post


- `url`: The URL that the posts links to (if the post has a URL)


- `num_points`: The number of points the post acquired (the total number of upvotes minus the total number of downvotes)


- `num_comments`: The number of comments that were made on the post


- `author`: The username of the person who submitted the post


- `created_at`: The date and time at which the post was submitted